# Logistic Regression 

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

## Loading dataset from MNIST

In [3]:
train_dataset = dsets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = dsets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

Processing...
Done!


In [4]:
(len(train_dataset))

60000

## Sampling

In [7]:
dset_size = len(train_dataset)
indices = list(range(dset_size))
train_end = math.floor(0.75*dset_size)
train_idx, valid_idx = indices[:train_end],indices[train_end:]
train_sample = SubsetRandomSampler(train_idx)
valid_sample = SubsetRandomSampler(valid_idx)

## Parameters

In [9]:
batch_size = 64
epochs = 2
learning_rate=0.01

In [11]:
train_features = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=train_sample)
valid_features = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=valid_sample)
test_features = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [12]:
print('train n_batches ',len(train_features))
print('valid n_batches ',len(valid_features))
print('test n_batches',len(test_features))

train n_batches  704
valid n_batches  235
test n_batches 157


## Defining model

In [16]:
class LogisticRegression(nn.Module):
    def __init__(self,input_size,num_classes):
        super(LogisticRegression,self).__init__()
        fc1_out_size = 64
        self.fc1 = nn.Linear(input_size,fc1_out_size)
        self.fc2 = nn.Linear(fc1_out_size,num_classes)        
    
    def forward(self,x):
        fc1 = self.fc1(x)
        fc1 = F.batch_norm(fc1)
        fc1 = F.softmax(fc1)
        fc2 = self.fc2(fc1)
        fc2 = F.batch_norm(fc2)
        return F.softmax(fc2)
        

In [18]:
model = LogisticRegression(input_size=28*28,num_classes=10)

## Defining Loss

In [32]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),learning_rate)

## Training model

In [31]:
iterations = 0
for epoch in range(epochs):
    for i,(images,labels) in enumerate(train_features):
        
        '''
        As long as the image shape is a 4D tensor(64,1,28,28) we need to reshape it to fit into model which
        is a linear fc with 28*28 input dim size. So the feature dim is (batch_size,28*28).
        '''
        features = Variable(images.view(-1,28*28))
        targets  = Variable(labels)

        optimizer.zero_grad()
        outputs = model(feature)
        loss = loss_function(outputs,targets)
        
        #backprop
        loss.backward()
        
        #step
        optimizer.step()

torch.Size([64]) tensor([ 8,  1,  5,  4,  0,  6,  1,  3,  3,  8,  4,  8,  6,  7,
         3,  9,  1,  9,  6,  6,  7,  4,  4,  9,  7,  4,  4,  6,
         5,  2,  7,  8,  7,  7,  8,  9,  0,  4,  3,  0,  6,  3,
         6,  6,  3,  1,  0,  4,  0,  8,  8,  3,  8,  3,  3,  7,
         1,  2,  8,  3,  9,  4,  6,  4])
torch.Size([64]) tensor([ 1,  6,  3,  8,  2,  5,  8,  4,  6,  3,  7,  7,  9,  7,
         1,  9,  5,  2,  3,  4,  9,  2,  3,  9,  0,  2,  2,  3,
         7,  4,  3,  6,  2,  8,  8,  3,  2,  9,  0,  7,  5,  5,
         0,  5,  5,  1,  2,  5,  7,  3,  7,  6,  7,  7,  9,  3,
         2,  3,  8,  0,  9,  1,  6,  1])
